In [26]:
!pip install transformers torch pandas -q

In [27]:
import torch
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import pipeline

In [30]:
train_df = pd.read_csv('C:\\Users\\Laptop World\\Desktop\\Sentiment Analysis\\twitter_validation.csv', header=None, names=['tweet_id', 'entity', 'sentiment', 'text'])

In [31]:
print("✅ حجم بيانات التدريب:", train_df.shape)
train_df.head()

✅ حجم بيانات التدريب: (1000, 4)


,tweet_id,entity,sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [33]:
val_df = pd.read_csv("C:\\Users\\Laptop World\\Desktop\\Sentiment Analysis\\twitter_validation.csv")  # غيّري الاسم حسب ملفك
print("✅ حجم بيانات الـ validation:", val_df.shape)
val_df.head()

✅ حجم بيانات الـ validation: (999, 4)


,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [36]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

C:\Users\Laptop World\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Laptop World\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [35]:
pip install tensorflow tf-keras

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
# عرض أول 5 صفوف من بيانات التحقق
print("أول 5 صفوف من بيانات التحقق:")
print(validation_df.head())

أول 5 صفوف من بيانات التحقق:
   tweet_id     entity   sentiment  \
0      3364   Facebook  Irrelevant   
1       352     Amazon     Neutral   
2      8312  Microsoft    Negative   
3      4371      CS-GO    Negative   
4      4433     Google     Neutral   

                                                text  \
0  I mentioned on Facebook that I was struggling ...   
1  BBC News - Amazon boss Jeff Bezos rejects clai...   
2  @Microsoft Why do I pay for WORD when it funct...   
3  CSGO matchmaking is so full of closet hacking,...   
4  Now the President is slapping Americans in the...   

                                        cleaned_text  sentiment_encoded  
0  i mentioned on facebook that i was struggling ...                  3  
1  bbc news  amazon boss jeff bezos rejects claim...                  1  
2  microsoft why do i pay for word when it functi...                  0  
3  csgo matchmaking is so full of closet hacking ...                  0  
4  now the president is slapping am

In [8]:
# عرض معلومات عن البيانات
print("\nمعلومات عن بيانات التحقق:")
print(validation_df.info())


معلومات عن بيانات التحقق:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   1000 non-null   int64 
 1   entity     1000 non-null   object
 2   sentiment  1000 non-null   object
 3   text       1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB
None


In [9]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[%s]' % re.escape(r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)
    return text

In [21]:
df.dropna(subset=['text', 'sentiment'], inplace=True)

In [22]:
df['cleaned_text'] = validation_df['text'].apply(clean_text)

In [23]:
print("\nأول 5 نصوص بعد التنظيف في بيانات التحقق:")
print(validation_df[['text', 'cleaned_text']].head())


أول 5 نصوص بعد التنظيف في بيانات التحقق:
                                                text  \
0  I mentioned on Facebook that I was struggling ...   
1  BBC News - Amazon boss Jeff Bezos rejects clai...   
2  @Microsoft Why do I pay for WORD when it funct...   
3  CSGO matchmaking is so full of closet hacking,...   
4  Now the President is slapping Americans in the...   

                                        cleaned_text  
0  i mentioned on facebook that i was struggling ...  
1  bbc news  amazon boss jeff bezos rejects claim...  
2  microsoft why do i pay for word when it functi...  
3  csgo matchmaking is so full of closet hacking ...  
4  now the president is slapping americans in the...  


In [24]:
# تحويل تصنيفات المشاعر إلى أرقام بنفس الترتيب السابق
sentiment_mapping = {'Positive': 2, 'Negative': 0, 'Neutral': 1, 'Irrelevant': 3}
validation_df['sentiment_encoded'] = validation_df['sentiment'].map(sentiment_mapping)

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [38]:
df.columns = ["id", "entity", "sentiment", "text"]

In [39]:
df['cleaned_text'] = df['text'].str.lower()

In [40]:
le = LabelEncoder()
df['sentiment_encoded'] = le.fit_transform(df['sentiment'])

In [41]:
X = df['cleaned_text']
y = df['sentiment_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ جاهزين للتدريب!")
print(df.head())

✅ جاهزين للتدريب!
     id     entity   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                                text  \
0  I mentioned on Facebook that I was struggling ...   
1  BBC News - Amazon boss Jeff Bezos rejects clai...   
2  @Microsoft Why do I pay for WORD when it funct...   
3  CSGO matchmaking is so full of closet hacking,...   
4  Now the President is slapping Americans in the...   

                                        cleaned_text  sentiment_encoded  
0  i mentioned on facebook that i was struggling ...                  0  
1  bbc news - amazon boss jeff bezos rejects clai...                  2  
2  @microsoft why do i pay for word when it funct...                  1  
3  csgo matchmaking is so full of closet hacking,...                  1  
4  now the president is slapping americans in the...                  

In [48]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

Device set to use cpu


In [47]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [49]:
!pip install datasets

In [46]:
!pip install datasets fsspec==2025.3.2 --upgrade

INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.

  Attempting uninstall: dill

    Found existing installation: dill 0.3.8

    Uninstalling dill-0.3.8:

      Successfully uninstalled dill-0.3.8

  Attempting uninstall: multiprocess

   ------------- -------------------------- 1/3 [multiprocess]

In [50]:
# تحديد الجهاز (CPU أو GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📌 Device set to: {device}")

📌 Device set to: cpu


In [51]:
# تحميل البيانات (مثال: IMDB)
dataset = load_dataset("imdb")

ValueError: Invalid pattern: '**' can only be an entire path component

In [54]:
pip install huggingface_hub[hf_xet]

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 3.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.3/2.8 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [56]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"